In [0]:
import pyspark
import pyspark.pandas as pd     #podmieniam ps na pd - tak konwencja podpowiada, żeby pandasa zapisywać jako pd ~SC
file_format = "delta"
file_location = "https://pandemicdatalake.blob.core.windows.net/public/curated/covid-19/ecdc_cases/latest/ecdc_cases.parquet"

df2 = pd.read_delta(file_location)

---------------------------------------------------------------------------
UnsupportedOperationException             Traceback (most recent call last)
File <command-167292954290977>, line 6
      3 file_format = "delta"
      4 file_location = "https://pandemicdatalake.blob.core.windows.net/public/curated/covid-19/ecdc_cases/latest/ecdc_cases.parquet"
----> 6 df2 = pd.read_delta(file_location)

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/pandas/namespace.py:628, in read_delta(path, version, timestamp, index_col, **options)
    626 if timestamp is not None:
    627     options["timestampAsOf"] = timestamp
--> 628 return read_

In [0]:
from pyspark.sql import SparkSession

file_location = "https://pandemicdatalake.blob.core.windows.net/public/curated/covid-19/ecdc_cases/latest/ecdc_cases.parquet"

spark = SparkSession.builder.getOrCreate()
df = spark.read.format("delta").load(file_location)

---------------------------------------------------------------------------
UnsupportedOperationException             Traceback (most recent call last)
File <command-167292954290998>, line 6
      3 file_location = "https://pandemicdatalake.blob.core.windows.net/public/curated/covid-19/ecdc_cases/latest/ecdc_cases.parquet"
      5 spark = SparkSession.builder.getOrCreate()
----> 6 df = spark.read.format("delta").load(file_location)

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:312, in DataFrameReader.load(self, path, format, schema, **options)
    310 self.options(**options)
    311 if isinstance(path, str):


In [0]:
#%pip install openpyxl 
import pandas as pd
import openpyxl # Q1 - how to make it work, even when library is not install and run quickly that check

# reference path the input file that was recently uploaded
file_path = "/dbfs/FileStore/tables/stockwatch_20231230-1.xlsx"
df = pd.read_excel(file_path)

#df.head()

# change column names
df.rename(columns = {'tablescraper-selected-row':'Spółka',
                     'tablescraper-selected-row href':'Rekomendacje wydane dla Spółki (URL)',
                     'r':'Cena docelowa', 
                     'r 2':'Obecny kurs',
                     'c':'Data wydania rekomendacji',
                     'r 3':'Kurs w dniu wydania',
                     'c 2':'Instytucja',
                     'c href':'Rekomendacje Domu Maklerskiego (URL)',
                     'btn href':'Raportu (URL)',
                     'c 3':'Rekomendacja'
                     }, inplace = True) 

#df[df['Rekomendacja'].notnull()]

# remove empty rows with "NaN" values from Columns
df = df.dropna()

df['Date']= pd.to_datetime(df['Date'])

# print top5 results
df.head()

# print schema
#df.info(verbose=True)  


,id,Spółka,Rekomendacje wydane dla Spółki (URL),Cena docelowa,Obecny kurs,Data wydania rekomendacji,Kurs w dniu wydania,Instytucja,Rekomendacje Domu Maklerskiego (URL),Raportu (URL),Rekomendacja
12,13,CCC,"https://www.stockwatch.pl/gpw/ccc,rekomendacje...","69,50 zł","61,18",2023-12-11,"58,34 zł",DM BOŚ SA,https://www.stockwatch.pl/rekomendacje/dm-bos-...,https://www.stockwatch.pl/recommendations/2023...,kupuj
23,24,ERBUD,"https://www.stockwatch.pl/gpw/erbud,rekomendac...","43,20 zł","42,80",2023-12-11,"35,20 zł",DM BOŚ SA,https://www.stockwatch.pl/rekomendacje/dm-bos-...,https://www.stockwatch.pl/recommendations/2023...,kupuj
24,25,BUDIMEX,"https://www.stockwatch.pl/gpw/budimex,rekomend...","490,00 zł","621,00",2023-12-11,"580,00 zł",DM BOŚ SA,https://www.stockwatch.pl/rekomendacje/dm-bos-...,https://www.stockwatch.pl/recommendations/2023...,sprzedaj
29,30,PGE,"https://www.stockwatch.pl/gpw/pge,rekomendacje...","11,00 zł","8,65",2023-12-11,"8,94 zł",DM BOŚ SA,https://www.stockwatch.pl/rekomendacje/dm-bos-...,https://www.stockwatch.pl/recommendations/2023...,neutralnie
30,31,PKNORLEN,"https://www.stockwatch.pl/gpw/pknorlen,rekomen...","99,00 zł","65,68",2023-12-11,"63,37 zł",DM BOŚ SA,https://www.stockwatch.pl/rekomendacje/dm-bos-...,https://www.stockwatch.pl/recommendations/2023...,neutralnie


In [0]:
import pandas as pd
import openpyxl
from pyspark.sql import SparkSession

# Create a Spark Session
spark = SparkSession.builder.getOrCreate()

# Reference the path to the input file that was recently uploaded
file_path = "/dbfs/FileStore/tables/stockwatch_20231230-1.xlsx"
df = pd.read_excel(file_path)

# Convert pandas DataFrame to Spark DataFrame
spark_df = spark.createDataFrame(df)

# Change column names
spark_df = spark_df.withColumnRenamed('tablescraper-selected-row', 'Spółka') \
    .withColumnRenamed('tablescraper-selected-row href', 'Rekomendacje wydane dla Spółki (URL)') \
    .withColumnRenamed('r', 'Cena docelowa') \
    .withColumnRenamed('r 2', 'Obecny kurs') \
    .withColumnRenamed('c', 'Data wydania rekomendacji') \
    .withColumnRenamed('r 3', 'Kurs w dniu wydania') \
    .withColumnRenamed('c 2', 'Instytucja') \
    .withColumnRenamed('c href', 'Rekomendacje Domu Maklerskiego (URL)') \
    .withColumnRenamed('btn href', 'Raportu (URL)') \
    .withColumnRenamed('c 3', 'Rekomendacja')

# Filter rows where 'Rekomendacja' is not null
filtered_df = spark_df.filter(spark_df['Rekomendacja'].isNotNull())

# Show the filtered DataFrame
filtered_df.show()

# Print top 5 rows
filtered_df.show(5)

# Print the schema
filtered_df.printSchema()

+---+---------+------------------------------------+-------------+-----------+-------------------------+-------------------+--------------------+------------------------------------+--------------------+------------+
| id|   Spółka|Rekomendacje wydane dla Spółki (URL)|Cena docelowa|Obecny kurs|Data wydania rekomendacji|Kurs w dniu wydania|          Instytucja|Rekomendacje Domu Maklerskiego (URL)|       Raportu (URL)|Rekomendacja|
+---+---------+------------------------------------+-------------+-----------+-------------------------+-------------------+--------------------+------------------------------------+--------------------+------------+
| 13|      CCC|                https://www.stock...|     69,50 zł|      61,18|               2023-12-11|           58,34 zł|           DM BOŚ SA|                https://www.stock...|https://www.stock...|       kupuj|
| 24|    ERBUD|                https://www.stock...|     43,20 zł|      42,80|               2023-12-11|           35,20 zł|        

In [0]:
%sql
-- Load Parquet file into DataFrame
CREATE OR REPLACE TEMPORARY VIEW covid19_temp
USING parquet
OPTIONS (path "https://pandemicdatalake.blob.core.windows.net/public/curated/covid-19/ecdc_cases/latest/ecdc_cases.parquet");

SELECT * FROM covid19_temp LIMIT 10;

-- Create DLT from DataFrame
CREATE OR REFRESH LIVE TABLE covid19 AS SELECT * FROM covid19_temp;

-- Create DLT for filtered data
CREATE OR REFRESH LIVE TABLE filtered_data
AS SELECT *
FROM LIVE.covid19;

com.databricks.backend.common.rpc.SparkDriverExceptions$SQLExecutionException: java.lang.UnsupportedOperationException
	at org.apache.hadoop.fs.http.AbstractHttpFileSystem.listStatus(AbstractHttpFileSystem.java:95)
	at org.apache.hadoop.fs.http.HttpsFileSystem.listStatus(HttpsFileSystem.java:23)
	at org.apache.spark.sql.execution.datasources.InMemoryFileIndex$.listFiles(InMemoryFileIndex.scala:442)
	at org.apache.spark.sql.execution.datasources.InMemoryFileIndex$.listLeafFiles(InMemoryFileIndex.scala:493)
	at org.apache.spark.sql.execution.datasources.InMemoryFileIndex$.$anonfun$bulkListLeafFiles$5(InMemoryFileIndex.scala:328)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at scala.collection.generic.Growable.$plus$plus$eq(Growable.scala:62)
	at scala.collection.generic.Growable.$plus$pl